#RASPADO DE COMPONENTES HISTÓRICOS S&P 500

In [1]:
# Import the necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Define the URL of the Wikipedia page for scraping
URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Function to make the HTTP request and obtain the content
def get_html_content(url):
    response = requests.get(url)
    # Confirm that the request was successful (HTTP status code 200)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Error in obtaining page content, status code: {response.status_code}")

# Function to parse HTML content with BeautifulSoup
def parse_html_to_soup(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    # Confirm that the BeautifulSoup object was created
    return soup

# Get the HTML content of the Wikipedia page
html_content = get_html_content(URL)

# Parse the HTML content to get the BeautifulSoup object
soup = parse_html_to_soup(html_content)

# Verify that significant content was extracted
# To do this, look for a distinctive element of the page that we know should be present
test_element = soup.find('h1', id="firstHeading")
assert test_element is not None, "The HTML content does not contain the expected element."

print("The HTML content has been successfully extracted and parsed.")


The HTML content has been successfully extracted and parsed.


In [2]:
# Function to extract the data from the current S&P 500 constituents table
def extract_current_snp_data(soup):
    table = soup.find('table', {'id': 'constituents'})
    rows = table.find_all('tr')[1:]  # Exclude the table header

    current_data = []
    for row in rows:
        cols = row.find_all('td')
        security = cols[1].text.strip()
        symbol = cols[0].text.strip()
        current_data.append({
            'Security': security,
            'Symbol': symbol,
            'Date': '1900-01-01'  # Fixed date for original members
        })

    return current_data

# Function to extract historical changes data
def extract_historical_changes_data(soup):
    table = soup.find('table', {'id': 'changes'})
    rows = table.find_all('tr')[1:]  # Exclude the table header

    historical_data = []
    for i, row in enumerate(rows):
        cols = row.find_all('td')
        if len(cols) < 6:  # Ensure there are enough columns
            print(f"Row {i+1} skipped: does not contain enough columns.")
            continue

        # Parse the date and ensure it's in the correct format
        date_text = cols[0].text.strip()
        try:
            date = datetime.strptime(date_text, '%B %d, %Y').strftime('%Y-%m-%d')
        except ValueError:
            print(f"Error parsing the date in row {i+1}: {date_text}")
            continue

        # Extract symbols and securities for added and removed entries
        added_symbol = cols[1].text.strip()
        added_security = cols[2].text.strip()
        removed_symbol = cols[3].text.strip()
        removed_security = cols[4].text.strip()

        # Add to the DataFrame the added and removed values as separate entries
        historical_data.append({'Security': added_security, 'Symbol': added_symbol, 'Date': date, 'Change': 'Added'})
        historical_data.append({'Security': removed_security, 'Symbol': removed_symbol, 'Date': date, 'Change': 'Removed'})

    return historical_data

# Extract data and create DataFrames
current_data = extract_current_snp_data(soup)
historical_data = extract_historical_changes_data(soup)

dataframe_current = pd.DataFrame(current_data)
dataframe_historical = pd.DataFrame(historical_data)

# Display the first records to confirm
print(dataframe_current.head())
print(dataframe_historical.head())



Row 1 skipped: does not contain enough columns.
      Security Symbol        Date
0           3M    MMM  1900-01-01
1  A. O. Smith    AOS  1900-01-01
2       Abbott    ABT  1900-01-01
3       AbbVie   ABBV  1900-01-01
4    Accenture    ACN  1900-01-01
              Security Symbol        Date   Change
0              Hubbell   HUBB  2023-10-18    Added
1        Organon & Co.    OGN  2023-10-18  Removed
2  Lululemon Athletica   LULU  2023-10-18    Added
3  Activision Blizzard   ATVI  2023-10-18  Removed
4                              2023-10-03    Added


## Datos faltantes

In [3]:
# Revisar datos faltantes en la columna 'Symbol' para el DataFrame de componentes actuales
missing_symbols_current = dataframe_current['Symbol'].isnull().sum()
print("Datos faltantes en la columna 'Symbol' para los componentes actuales del S&P 500:")
print(missing_symbols_current)
print("\nFilas con datos faltantes en la columna 'Symbol' para los componentes actuales del S&P 500:")
print(dataframe_current[dataframe_current['Symbol'].isnull()])

# Revisar datos faltantes en la columna 'Symbol' para el DataFrame de cambios históricos
missing_symbols_historical = dataframe_historical['Symbol'].isnull().sum()
print("\nDatos faltantes en la columna 'Symbol' para los cambios históricos del S&P 500:")
print(missing_symbols_historical)
print("\nFilas con datos faltantes en la columna 'Symbol' para los cambios históricos del S&P 500:")
print(dataframe_historical[dataframe_historical['Symbol'].isnull()])

# Identificar símbolos que son cadenas vacías o solo espacios en el DataFrame de componentes actuales
empty_symbols_current = dataframe_current[dataframe_current['Symbol'].str.strip() == '']
print("Símbolos vacíos en los componentes actuales del S&P 500:")
print(empty_symbols_current)

# Identificar símbolos que son cadenas vacías o solo espacios en el DataFrame de cambios históricos
empty_symbols_historical = dataframe_historical[dataframe_historical['Symbol'].str.strip() == '']
print("\nSímbolos vacíos en los cambios históricos del S&P 500:")
print(empty_symbols_historical)



Datos faltantes en la columna 'Symbol' para los componentes actuales del S&P 500:
0

Filas con datos faltantes en la columna 'Symbol' para los componentes actuales del S&P 500:
Empty DataFrame
Columns: [Security, Symbol, Date]
Index: []

Datos faltantes en la columna 'Symbol' para los cambios históricos del S&P 500:
0

Filas con datos faltantes en la columna 'Symbol' para los cambios históricos del S&P 500:
Empty DataFrame
Columns: [Security, Symbol, Date, Change]
Index: []
Símbolos vacíos en los componentes actuales del S&P 500:
Empty DataFrame
Columns: [Security, Symbol, Date]
Index: []

Símbolos vacíos en los cambios históricos del S&P 500:
    Security Symbol        Date   Change
4                    2023-10-03    Added
7                    2023-10-02  Removed
24                   2023-01-05    Added
27                   2023-01-04  Removed
32                   2022-12-19    Added
35                   2022-12-15  Removed
48                   2022-06-21    Added
62                  

In [4]:
# Eliminar filas donde la columna 'Symbol' contiene una cadena vacía o solo espacios en blanco
dataframe_historical_cleaned = dataframe_historical[dataframe_historical['Symbol'].str.strip() != '']

# Verificar que las filas han sido eliminadas
print(dataframe_historical_cleaned)


                          Security Symbol        Date   Change
0                          Hubbell   HUBB  2023-10-18    Added
1                    Organon & Co.    OGN  2023-10-18  Removed
2              Lululemon Athletica   LULU  2023-10-18    Added
3              Activision Blizzard   ATVI  2023-10-18  Removed
5                   DXC Technology    DXC  2023-10-03  Removed
..                             ...    ...         ...      ...
663                     General Re    GRN  1998-12-11  Removed
664                      Compuware   CPWR  1998-12-11    Added
665                     SunAmerica    SUN  1998-12-11  Removed
666  Countrywide Credit Industries    CCI  1997-06-17    Added
667                         USLife    USL  1997-06-17  Removed

[641 rows x 4 columns]


In [5]:

# Identificar símbolos que son cadenas vacías o solo espacios en el DataFrame de cambios históricos
empty_symbols_historical = dataframe_historical_cleaned[dataframe_historical_cleaned['Symbol'].str.strip() == '']
print("\nSímbolos vacíos en los cambios históricos del S&P 500:")
print(empty_symbols_historical)



Símbolos vacíos en los cambios históricos del S&P 500:
Empty DataFrame
Columns: [Security, Symbol, Date, Change]
Index: []


In [6]:
# Guardar el DataFrame de componentes actuales del S&P 500 en un archivo CSV
dataframe_current.to_csv('current_snp500.csv', index=False)

# Guardar el DataFrame de cambios históricos del S&P 500 limpio en un archivo CSV
dataframe_historical_cleaned.to_csv('historical_changes_snp500.csv', index=False)



## Crear base de consulta (Si el stock pertenecía o no al S&P)

In [11]:
# Step 1: Combine and Prepare DataFrames
def preparar_dataframes(dataframe_current, dataframe_historical):
    start_date = pd.to_datetime('1900-01-01')
    dataframe_current['Date'] = start_date
    dataframe_current['Change'] = 'Added'

    # Create a copy of the added data from the historical dataframe
    added_historical = dataframe_historical[dataframe_historical['Change'] == 'Added'].drop_duplicates(subset='Symbol', keep='first')

    # Update the current dataframe with the addition data from historical
    for index, row in added_historical.iterrows():
        current_index = dataframe_current[dataframe_current['Symbol'] == row['Symbol']].index
        if not current_index.empty:
            dataframe_current.loc[current_index, 'Date'] = row['Date']

    combined_dataframe = pd.concat([dataframe_current, dataframe_historical]).drop_duplicates(subset=['Symbol', 'Date', 'Change'], keep='last')
    combined_dataframe.sort_values(by=['Symbol', 'Date'], inplace=True)

    return combined_dataframe, added_historical

# Step 2: Create Stock Status Dictionary
def crear_diccionario_estados(combined_dataframe, added_historical):
    statuses = {}
    start_date = pd.to_datetime('1900-01-01')
    for _, row in combined_dataframe.iterrows():
        if row['Symbol'] not in statuses:
            statuses[row['Symbol']] = []
            # If the stock was removed but has no addition date, assign the start date
            if row['Change'] == 'Removed' and row['Symbol'] not in added_historical['Symbol'].values:
                statuses[row['Symbol']].append((start_date, 'Added'))
        statuses[row['Symbol']].append((row['Date'], row['Change']))
    return statuses

# Step 3: Function to Check Membership
def verificar_pertenencia(symbol, date, statuses):
    date = pd.to_datetime(date)  # Ensure 'date' is a Timestamp
    changes = statuses.get(symbol, [])

    was_member = False
    for change_date, change_status in sorted(changes, key=lambda x: pd.to_datetime(x[0])):
        if pd.to_datetime(change_date) > date:
            break
        was_member = change_status == 'Added'
    return was_member

# Step 4: Function to Process Multiple Queries
def procesar_consultas(queries, statuses):
    results = []
    for symbol, date in queries:
        belongs = verificar_pertenencia(symbol, date, statuses)
        results.append((symbol, date, belongs))
    return results

# Prepare DataFrames and create the status dictionary
combined_dataframe, added_historical = preparar_dataframes(dataframe_current, dataframe_historical)
statuses = crear_diccionario_estados(combined_dataframe, added_historical)

# Example usage
queries = [("HPH", "1999-01-01"), ("HPH", "2000-01-01")]
results = procesar_consultas(queries, statuses)
for symbol, date, belongs in results:
    print(f"The symbol {symbol} {'was' if belongs else 'was not'} a member of the S&P 500 on {date}.")


The symbol HPH was a member of the S&P 500 on 1999-01-01.
The symbol HPH was not a member of the S&P 500 on 2000-01-01.


<ipython-input-11-a989689e4e6c>:17: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  combined_dataframe.sort_values(by=['Symbol', 'Date'], inplace=True)
